In [1]:
#Import libraries
import time
import nltk
import string
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


c:\Users\irene\anaconda3\envs\SIC\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\irene\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irene\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\irene\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#Load data
DKdf_LinearSVR = pd.read_csv('..//Data_output//DKdf_LinearSVR.csv', delimiter=',')
DKdf_LinearSVR_sample = DKdf_LinearSVR[0:1000]

In [5]:
#Preproces text
def preprocess_text(text):
    if isinstance(text, str): 
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in string.punctuation]
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
        return ' '.join(tokens) 
    else:
        return ''

DKdf_LinearSVR_sample['Ingredients'] = DKdf_LinearSVR_sample['Ingredients'].apply(preprocess_text)

DKdf_LinearSVR_sample['ingredients_tokenized'] = DKdf_LinearSVR_sample['Ingredients'].apply(lambda x: set(x.split()))

C:\Users\irene\AppData\Local\Temp\ipykernel_6000\68057289.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DKdf_LinearSVR_sample['Ingredients'] = DKdf_LinearSVR_sample['Ingredients'].apply(preprocess_text)
C:\Users\irene\AppData\Local\Temp\ipykernel_6000\68057289.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DKdf_LinearSVR_sample['ingredients_tokenized'] = DKdf_LinearSVR_sample['Ingredients'].apply(lambda x: set(x.split()))


In [19]:
#Add nan to do the evaluation
DKdf_LinearSVR_sample_nan = DKdf_LinearSVR_sample.copy()
DKdf_LinearSVR_sample_nan['Predicted_Score'] = float('nan')
DKdf_LinearSVR_sample_nan

,url,vegetarianrecipe,veganrecipe,title,cookingmethod,ingredientlist,cookingtimelist,Predicted_Score,ingredients_tokenized
0,www.kochbar.de/rezept/289074/Grundrezept-fuer-...,1,0,Basic recipe for Spaetzle,"boil, boil, simmer",beet water onion salt lemon juic white pepper ...,-0.059511,NaN,"{juic, honey, sour, lemon, cream, water, white..."
1,www.kochbar.de/rezept/297292/Backwaren-Eierlik...,1,0,Baked goods egglikoer bananas Gugelhupf,"heat, bake",flour milk sugar vanilla salt egg french bread...,-0.063877,NaN,"{sugar, flour, mapl, french, cream, nut, powde..."
2,https://www.allrecipes.com/recipe/212065/sage-...,0,0,Sage Sausageand Cider Stuffing,"bake, cook",golden delici appl appl butter celeri chicken ...,-0.068242,NaN,"{sage, italian, delici, ½, bread, oliv, water,..."
3,NaN,1,0,Banana nut bread,bake,banana butter crème fraîch date egg pecan salt...,-0.050781,NaN,"{banana, flour, wheat, egg, pecan, date, butte..."
4,www.kochbar.de/rezept/245294/Aladins-Wunder-Sa...,1,1,Aladin's miracle salad,"boil, chill",potato celeri white onion hard-boil egg miracl...,-0.066059,NaN,"{sugar, dijon, fresh, potato, whip, salt, egg,..."
...,...,...,...,...,...,...,...,...,...
995,www.kochbar.de/rezept/240327/Bulgursalat-nach-...,1,1,Bulgur salad according to elves,NaN,romain lettuc feta chee pepperoncini pepper re...,-0.079156,NaN,"{romain, red, lettuc, pepperoncini, oliv, blac..."
996,https://www.allrecipes.com/recipe/235989/cream...,0,0,Creamy Ham and Bean Soup (Gluten Free),"heat, simmer, steam",yukon gold potato babi carrot butter celeri ch...,-0.067369,NaN,"{cauliflow, potato, butter, gold, milk, onion,..."
997,www.kochbar.de/rezept/180397/Giotto-Nusstorte-...,0,0,Giotto nut cake with peaches,bake,yellow cake mix water cream chee powder sugar ...,-0.059511,NaN,"{sugar, yellow, cream, powder, mix, egg, water..."
998,www.kochbar.de/rezept/121931/Safranreis-mit-ge...,1,0,Saffran rice with a busy gem,"steam, heat, heat, stir-fry, stir-fry",veget oil white rice egg green onion garlic cl...,-0.076973,NaN,"{veget, clove, carrot, kernel, sauc, soy, pea,..."


In [20]:
#Jaccard similarity
def jaccard(df1, df2):
    results = []
    for index, row in tqdm(df1[df1['Predicted_Score'].isna()].iterrows(), total=len(df1[df1['Predicted_Score'].isna()])):        
        #Get recipe name from DHLabdata
        ingredients = row['Ingredients']
        #Calculate Jaccard similarity between recipe names
        jaccard_similarities = [(fuzz.token_set_ratio(ingredients, name), name) for name in df2['Ingredients']]
        #Find recipe name with the highest Jaccard similarity
        closest_match_tuple = max(jaccard_similarities)
        #Extract the closest match from the tuple
        closest_match = closest_match_tuple[1]
        #Get index of closest match
        closest_index = df2[df2['Ingredients'] == closest_match].index[0]
        #Store the matched data
        result_row = {
            'Predicted_Score': df2.at[closest_index, 'Predicted_Score'] if pd.isna(row['Predicted_Score']) else row['Predicted_Score']
        }
        results.append(result_row)
        

    return pd.DataFrame(results)

#Print time
start_time = time.time()
DHLdf_jaccard = jaccard(DKdf_LinearSVR_sample_nan, DKdf_LinearSVR_sample)
end_time = time.time()
time_taken = end_time - start_time
print("Time taken:", time_taken)

#mae, mse, rmse
mae = mean_absolute_error(DKdf_LinearSVR_sample['Predicted_Score'], DHLdf_jaccard['Predicted_Score'])
mse = mean_squared_error(DKdf_LinearSVR_sample['Predicted_Score'], DHLdf_jaccard['Predicted_Score'])
rmse = mse ** 0.5
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

100%|██████████| 1000/1000 [06:10<00:00,  2.70it/s]

Time taken: 370.51214385032654
Mean Absolute Error (MAE) for time_to_make: 1.9689802726582677
Mean Squared Error (MSE) for time_to_make: 17.471979767233744
Root Mean Squared Error (RMSE) for time_to_make: 4.179949732620447


In [22]:
#Cosine similarity
def cosine_score(text1, text2):
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform([text1, text2])
    cosine_similarities = cosine_similarity(count_matrix[0:1], count_matrix)
    return cosine_similarities[0][1]

def cosine(df1, df2):
    results = []
    for index, row in tqdm(df1.iterrows(), total=len(df1)):
        # Get recipe name from DHLabdata
        recipe_name = row['Ingredients']
        if pd.isna(recipe_name):
            continue
        # Calculate cosine similarities between recipe names
        cosine_similarities = [(
            cosine_score(recipe_name, name),
            name
        ) for name in df2['Ingredients']]
        # Find recipe name with the highest cosine similarity
        closest_match_tuple = max(cosine_similarities)
        # Extract the closest match from the tuple
        closest_match = closest_match_tuple[1]
        # Get index of closest match
        closest_index = df2[df2['Ingredients'] == closest_match].index[0]
        # Store the matched data
        result_row = {
            'Predicted_Score': df2.at[closest_index, 'Predicted_Score'] if pd.isna(row['Predicted_Score']) else row['Predicted_Score']
        }
        results.append(result_row)
    return pd.DataFrame(results)

#Print time
start_time = time.time()
DHLdf_cosine = cosine(DKdf_LinearSVR_sample_nan, DKdf_LinearSVR_sample)
end_time = time.time()
time_taken = end_time - start_time
print("Time taken:", time_taken)

#mae, mse, rmse
mae = mean_absolute_error(DKdf_LinearSVR_sample['Predicted_Score'], DHLdf_cosine['Predicted_Score'])
mse = mean_squared_error(DKdf_LinearSVR_sample['Predicted_Score'], DHLdf_cosine['Predicted_Score'])
rmse = mse ** 0.5
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)


100%|██████████| 1000/1000 [16:18<00:00,  1.02it/s]

Time taken: 978.8358137607574
Mean Absolute Error (MAE) for time_to_make: 1.6482595275443641
Mean Squared Error (MSE) for time_to_make: 12.898780680606091
Root Mean Squared Error (RMSE) for time_to_make: 3.5914872519063867
